In [31]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder 


In [32]:
excel_file_path = 'fyp_test.xlsx'

xls = pd.ExcelFile(excel_file_path)
sheet_names = xls.sheet_names

print("Sheet Names:", sheet_names)

excel_sheets = pd.read_excel(excel_file_path, sheet_name=None, header=None)

column_info = []
for sheet_name, df in excel_sheets.items():
    columns = df.iloc[0].tolist()
    
    for i in columns:
        if i not in column_info:
            column_info.append(i)
column_info.append("categories")
maindf =  pd.DataFrame(columns=column_info)


Sheet Names: ['A&E Service Fees', 'Abdominal Binder', 'Abiraterone', 'Acarbose', 'Access Instument', 'Acetazolamide', 'Acetic Acid', 'Acetone', 'Acetylcysteine', 'Aciclovir', 'Adalimumab', 'Adapalene', 'Ademetionine', 'Adenosine', 'Adhesive Remover', 'Admin Fee', 'Admission Processing Fee', 'Adrenaline', 'Afatinib dimaleate', 'Aflibercept', 'Agomelatine', 'Airway', 'Airway Connector', 'Albumin', 'Alcohol Solution alone', 'Alendronic Acid', 'Alfacalcidol', 'Alfentanil', 'Allopurinol', 'Alpha-lipoic acid', 'Alprazolam', 'Alteplase', 'Aluminium HYD+Mg Tri+Simethicon', 'Alverine citrate + Simethicone', 'Ambroxol', 'Ambulance Service', 'Amikacin', 'Amiloride', 'Amiodarone', 'Amitriptyline', 'Amlodipine', 'Amlodipine + Valsartan', 'Amlodipine+Valsartan+HCT', 'Amoxicillin', 'Amoxicillin + Clavulanic Acid', 'Ampicillin', 'Anaesthesia Fees', 'Anastrazole', 'Anchoring Device', 'Anidulafungin', 'Anti-D Immunoglobulins', 'Anti-fog Soln', 'Antithymocyte Immunoglobulin', 'Apixaban', 'Appetite Enhanc

In [33]:
def rename_volume_columns(df):
    volume_count = 1
    new_columns = []
    for column in df.columns:
        if 'volume' in column:
            new_column_name = f'bolume_{volume_count}'
            volume_count += 1
        else:
            new_column_name = column
        new_columns.append(new_column_name)
    df.columns = new_columns
    

In [34]:
all_sheets = pd.read_excel(excel_file_path, sheet_name=None)
combined_df = pd.DataFrame()

for sheet_name, sheet_df in all_sheets.items():
    sheet_df['Sheet_Name'] = sheet_name
    rename_volume_columns(sheet_df)
    try:
        combined_df = pd.concat([combined_df, sheet_df], ignore_index=True)
    except ValueError as e:
        print(f"Error processing sheet '{sheet_name}': {e}")

combined_df = combined_df[combined_df.columns.drop(list(combined_df.filter(regex='Unnamed')))]
combined_df = combined_df[combined_df.columns.drop(list(combined_df.filter(regex='Volume')))]
combined_df.columns = combined_df.columns.str.replace('bolume', 'Volume')


In [35]:
combined_df['PricingDate'] = pd.to_datetime(combined_df['PricingDate'])

combined_df['Year'] = combined_df['PricingDate'].dt.year
combined_df['tempmonth'] = combined_df['PricingDate'].dt.month
combined_df['tempday'] = combined_df['PricingDate'].dt.day

combined_df['Month'] = np.where((combined_df['tempmonth'] == 1) & (combined_df['tempday'] == 1), 1, np.maximum(combined_df['tempmonth'], combined_df['tempday']))



/var/folders/sp/84j13ybj0zd_p4y410zpx7jc0000gn/T/ipykernel_53419/3724801287.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  combined_df['Year'] = combined_df['PricingDate'].dt.year
/var/folders/sp/84j13ybj0zd_p4y410zpx7jc0000gn/T/ipykernel_53419/3724801287.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  combined_df['tempmonth'] = combined_df['PricingDate'].dt.month
/var/folders/sp/84j13ybj0zd_p4y410zpx7jc0000gn/T/ipykernel_53419/3724801287.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the res

In [36]:
def combinecol(df,x,y):
    df[x] = df[x].fillna('') + df[y].fillna('')

def catcoding(df,x):
    label_encoder = LabelEncoder()
    df["cat_" + x] = label_encoder.fit_transform(df[x])

In [37]:
combined_df['Procedure Name'] = combined_df[['Procedure Name', 'Surgery Name/Procedure Name']].apply(lambda x: ','.join(x.dropna()), axis=1)

#ApplicationMethod variable - merge multiple original variables together
merge_list = ['Delivery', 'Delivery Medium', 'Route', 'ROUTE', 'USAGE TYPE']

combined_df['ApplicationMethod'] = combined_df.loc[:, 'Delivery']
combined_df.head()

for col in merge_list:
    selected_col = combined_df[col]
    combined_df['ApplicationMethod'] = np.where(selected_col.replace('',np.nan).notna(),selected_col, combined_df['ApplicationMethod'])
merge_list1 = ['Body Part', 'Location']

combined_df['OperationArea'] = combined_df.loc[:, 'For Body Part']
combined_df.head()

for col in merge_list1:
    selected_col = combined_df[col]
    combined_df['OperationArea'] = np.where(selected_col.replace('',np.nan).notna(),selected_col, combined_df['OperationArea'])

#rename variables with more representative names
combined_df.rename(columns= {'Volume_1' : 'Vol_1st_med', 'Volume_2': 'Vol_2nd_med', 'Volume_3': 'Vol_3rd_med'} , inplace= 'True')

#merge variables together
# Disposible? and Disposible variables
combined_df['Disposable?'] = np.where(combined_df['Disposable'].replace('',np.nan).notna(),combined_df['Disposable'],combined_df['Disposable?'])
combined_df['Disposable?'] = np.where(combined_df['Diposable?'].replace('',np.nan).notna(),combined_df['Diposable?'],combined_df['Disposable?'])

#Surgery & prodecure name
combined_df['Surgical Procedure'] = np.where(combined_df['Surgery Name/Procedure Name'].replace('',np.nan).notna(),combined_df['Surgery Name/Procedure Name'],combined_df['Procedure Name'])

#convert variables into binary variables
combined_df['SpecificConsultant?'] = np.where(combined_df['Consultant'].isnull(), 'No', 'Yes')
combined_df['Disposable?'] = np.where(combined_df['Disposable?'].isnull(), 'No', 'Yes')
combined_df['Urgency?'] = np.where(combined_df['Urgency'].isnull(), 'No', 'Yes')
combined_df['Brand'] = np.where(combined_df['Brand '].replace('',np.nan).notna(),combined_df['Brand '],combined_df['Brand'])

#convert to cateogorical variables
combined_df['Sterile?'] = np.where(combined_df['Sterile?'].isnull(), 'NA', combined_df['Sterile?'])

#create new variable - care classification for binary - inpatient and outpatient
def in_out_care(df):
    
    if (df['Ward'] == 'Outpatient'):
        return 'Outpatient'
    elif (df['Ward'] == 'A&E'): #classification adapted from MOM 
        return 'Outpatient'
    else:
        return 'Inpatient'
    
combined_df['Care Classification'] = combined_df.apply(in_out_care, axis = 1)

combined_df.columns = combined_df.columns.str.replace(' ', '_')
#removecol = ['Consultant','Condition','QuantityNotMoreThan','Period','Product_Code',]

/var/folders/sp/84j13ybj0zd_p4y410zpx7jc0000gn/T/ipykernel_53419/965655139.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  combined_df['ApplicationMethod'] = combined_df.loc[:, 'Delivery']
/var/folders/sp/84j13ybj0zd_p4y410zpx7jc0000gn/T/ipykernel_53419/965655139.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  combined_df['OperationArea'] = combined_df.loc[:, 'For Body Part']
/var/folders/sp/84j13ybj0zd_p4y410zpx7jc0000gn/T/ipykernel_53419/965655139.py:30: PerformanceWarning: DataFrame is highly fragmented.  This is usu

In [38]:
combined_df = combined_df[combined_df['Year'] > 2016]

In [39]:
#grouped_data = combined_df.groupby(['Parent', 'Child'])

#groupeddata = grouped_data.filter(lambda x: len(x) >= 3)

#filtered_data = grouped_data.filter(lambda x: len(x['Year'].unique()) >= 3)

In [40]:
def has_consecutive_years(years):
    years_set = set(years)
    for year in years_set:
        if year + 1 in years_set and year + 2 in years_set:
            return True
    return False

# Group by child and check for consecutive years, then filter
filtered_df = combined_df.groupby(['Parent', 'Child']).filter(lambda x: has_consecutive_years(x['Year']) )

filtered_df = filtered_df.groupby(['Parent', 'Child']).filter(lambda x: len(x['Year'].unique()) >= 3)


In [41]:
filtered_data = filtered_df


In [42]:
threshold = len(filtered_data) * 0.1
df_cleaned = filtered_data.dropna(thresh=threshold, axis=1)
df_cleaned

,Code,Description,Type,Hospital,Ward,Consultant,PricingDate,Price,Minimum,Maximum,...,Test,Year,tempmonth,tempday,Month,ApplicationMethod,Surgical_Procedure,SpecificConsultant?,Urgency?,Care_Classification
0,n.a.,A&E Amount Transferred,AMOUNT TRANSFERRED,National University Hospital,A&E,NaN,2020-01-01,121.0,5,5,...,NaN,2020,1,1,1,NaN,,No,No,Outpatient
1,n.a.,A&E Amount Transferred,AMOUNT TRANSFERRED,National University Hospital,A&E,NaN,2019-01-11,121.0,5,5,...,NaN,2019,1,11,11,NaN,,No,No,Outpatient
2,n.a.,A&E Amount Transferred,AMOUNT TRANSFERRED,National University Hospital,A&E,NaN,2020-01-02,121.0,5,5,...,NaN,2020,1,2,2,NaN,,No,No,Outpatient
3,n.a.,A&E Amount Transferred,AMOUNT TRANSFERRED,National University Hospital,A&E,NaN,2020-01-03,121.0,5,5,...,NaN,2020,1,3,3,NaN,,No,No,Outpatient
4,n.a.,A&E Amount Transferred,AMOUNT TRANSFERRED,National University Hospital,A&E,NaN,2020-01-05,121.0,5,5,...,NaN,2020,1,5,5,NaN,,No,No,Outpatient
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46377,IMOV1,IMOVANE 7.5MG TABLETS,NaN,Mount Elizabeth Orchard,Outpatient,NaN,2017-01-05,2.12,5,5,...,NaN,2017,1,5,5,NaN,,No,No,Outpatient
46379,n.a.,Zopiclone 7.5mg Tablet,NaN,National University Hospital,HD Class A,NaN,2019-01-11,0.54,5,5,...,NaN,2019,1,11,11,NaN,,No,No,Inpatient
46380,n.a.,Zopiclone 7.5mg Tablet,NaN,National University Hospital,Class A1,NaN,2019-01-12,0.54,5,5,...,NaN,2019,1,12,12,NaN,,No,No,Inpatient
46381,n.a.,ZOPICLONE 7.5MG TABLET,NaN,National University Hospital,Outpatient,NaN,2019-01-12,0.55,5,5,...,NaN,2019,1,12,12,NaN,,No,No,Outpatient


In [43]:
df_cleaned['Price'] = pd.to_numeric(df_cleaned['Price'], errors='coerce')
hospital_price = df_cleaned.groupby(['Hospital', 'Year'])['Price'].mean().reset_index().sort_values(by=['Hospital', 'Year'])
ward_price = df_cleaned.groupby(['Ward', 'Year'])['Price'].mean().reset_index().sort_values(by=['Ward', 'Year'])
parent_price = df_cleaned.groupby(['Parent', 'Year'])['Price'].mean().reset_index().sort_values(by=['Parent', 'Year'])
child_price = df_cleaned.groupby(['Child', 'Year'])['Price'].mean().reset_index().sort_values(by=['Child', 'Year'])
sheet_name_price = df_cleaned.groupby(['Sheet_Name', 'Year'])['Price'].mean().reset_index().sort_values(by=['Sheet_Name', 'Year'])
procedure_name_price = df_cleaned.groupby(['Procedure_Name', 'Year'])['Price'].mean().reset_index().sort_values(by=['Procedure_Name', 'Year'])

/var/folders/sp/84j13ybj0zd_p4y410zpx7jc0000gn/T/ipykernel_53419/1683806229.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['Price'] = pd.to_numeric(df_cleaned['Price'], errors='coerce')


In [44]:
nan_count = df_cleaned.isna().sum().sort_values(ascending=False)

print("Count of NaNs in each column:")
print(nan_count)


Count of NaNs in each column:
Test                    38859
Route                   38552
ApplicationMethod       37944
Consultant              37202
Dosage                  36043
Form                    35821
Footnotes               35610
QuantityNotMoreThan     34105
Period                  34105
Condition               34104
Footnote                31883
Brand                   31061
Company                 30019
Type                    26671
Code                     1091
Maximum                     1
Parent                      0
Ward                        0
Urgency?                    0
SpecificConsultant?         0
Surgical_Procedure          0
Hospital                    0
Month                       0
tempday                     0
tempmonth                   0
Year                        0
Sterile?                    0
Child                       0
Procedure_Name              0
Disposable?                 0
PricingDate                 0
Price                       0
Minimum   

In [45]:
def concat_columns(row):
    return ' '.join(str(item) for item in row)


df_cleaned['NLP'] = df_cleaned[['Description', 'Type', 'Hospital','Child','Sheet_Name','Footnote']].apply(concat_columns, axis=1)

descp = df_cleaned['NLP'].tolist()
unique_desc = list(set(descp))

/var/folders/sp/84j13ybj0zd_p4y410zpx7jc0000gn/T/ipykernel_53419/2294111214.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['NLP'] = df_cleaned[['Description', 'Type', 'Hospital','Child','Sheet_Name','Footnote']].apply(concat_columns, axis=1)


In [46]:
cat = []

for i in unique_desc:
    s = i.split()
    if s[0] not in cat:
        cat.append(s[0])

In [47]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from collections import Counter
import string
import nltk
#nltk.download('punkt')
#nltk.download('wordnet')  # For stemming


def remove_punctuation(text):
    return ''.join([char for char in text if char not in string.punctuation])

porter = PorterStemmer()


def tokenize_and_stem(text):
    tokens = word_tokenize(text)
    stemmed_tokens = [porter.stem(token) for token in tokens if token.isalnum()]
    return stemmed_tokens


tokenized_and_stemmed_descriptions = df_cleaned['NLP'].apply(tokenize_and_stem)
all_words = [word for sublist in tokenized_and_stemmed_descriptions for word in sublist]


word_counts = Counter(all_words)


common_words = word_counts.most_common()


print(common_words[:100])


[('nan', 58554), ('mount', 23455), ('elizabeth', 23434), ('novena', 19852), ('of', 12257), ('and', 11692), ('hospit', 11487), ('orchard', 9378), ('fee', 8919), ('for', 8892), ('the', 7380), ('cancer', 6573), ('univers', 6457), ('nation', 6400), ('treatment', 6397), ('centr', 5813), ('use', 5582), ('lab', 5347), ('parkway', 4833), ('in', 4571), ('to', 4423), ('with', 3583), ('consult', 3390), ('indic', 3163), ('inject', 3075), ('gener', 3055), ('as', 3038), ('biochemistri', 2937), ('infect', 2864), ('blood', 2566), ('charg', 2525), ('tablet', 2513), ('a', 2473), ('x', 2368), ('other', 2325), ('dress', 2242), ('set', 2219), ('skin', 2198), ('servic', 2195), ('singapor', 2177), ('consum', 2168), ('gleneagl', 2025), ('tube', 2022), ('sodium', 1904), ('patient', 1871), ('steril', 1848), ('therapi', 1736), ('chlorid', 1720), ('diseas', 1681), ('or', 1674), ('is', 1641), ('transfus', 1600), ('medic', 1580), ('pain', 1535), ('nurs', 1450), ('daili', 1368), ('intraven', 1357), ('acut', 1344), (

In [48]:
get_excel_file = 'cancer_list.xlsx'
cancer_list = pd.read_excel(get_excel_file)
sub_cancer_list = cancer_list.copy()
cols = ['Cancer type', 'Active ingredient']
sub_cancer_list = sub_cancer_list[cols]
sub_cancer_list.dropna(subset=['Cancer type'], inplace=True)
cancer_word_list = []

for cancer_type, ingredient in sub_cancer_list.itertuples(index=False):
    if cancer_type not in cancer_word_list:
        cancer_word_list.append(cancer_type)
    elif ingredient not in cancer_word_list:
        cancer_word_list.append(ingredient)

cancer_word_list

['Breast cancer',
 'Abemaciclib',
 'Others',
 'Abiraterone',
 'Leukaemia',
 'Acalabrutinib',
 'Lymphoma',
 'Acalabrutinib plus obinutuzumab',
 'Lung cancer',
 'Colorectal cancer',
 'Alectinib',
 'Alpelisib',
 'Myeloproliferative neoplasms',
 'Anagrelide',
 'Anastrozole',
 'Prostate cancer',
 'Apalutamide',
 'Arsenic trioxide',
 'Liver cancer',
 'Atezolizumab',
 'Atezolizumab plus bevacizumab biosimilar',
 'Bladder cancer',
 'Atezolizumab plus nab-paclitaxel',
 'Atezolizumab plus bevacizumab',
 'Avelumab',
 'Merkel cell cancer',
 'Renal cancer',
 'Axitinib',
 'Azacitidine',
 'Bacillus Calmette-Guerin (BCG)',
 'Bendamustine',
 'Bevacizumab',
 'Bevacizumab biosimilar',
 'Bexarotene',
 'Bicalutamide',
 'Bleomycin',
 'Blinatumomab',
 'Bortezomib',
 'Brentuximab vedotin',
 'Brigatinib',
 'Bromocriptine',
 'Busulfan',
 'Cabazitaxel',
 'Neuroendocrine',
 'Cabozantinib',
 'Capecitabine',
 'Carboplatin',
 'Multiple myeloma',
 'Carfilzomib',
 'Carmustine',
 'Ceritinib',
 'Cetuximab',
 'Chlorambuc

In [49]:
file_path = "cancerdrugsdb.txt"

try:
    with open(file_path, 'r') as file:
        file_contents = file.read()
        splits = file_contents.split("\t")
        col_header = splits[0:14]
        datas = splits[14: len(splits)]

except:
    print("")

In [50]:

num_columns = len(col_header)
num_rows = len(datas) // num_columns
listB_reshaped = [datas[i:i+num_columns] for i in range(0, len(datas), num_columns)]

# Create DataFrame
df = pd.DataFrame(listB_reshaped, columns=col_header)

for i in df["Product"]:
    if i not in cancer_word_list:
        cancer_word_list.append(i)


In [51]:
my_list = df['Indications'].tolist()
filtered_list = [x for x in my_list if x is not None]

for i in filtered_list:
    if ";" in i:
        splits = i.split(";")
        for j in splits:
            if j not in cancer_word_list:
                cancer_word_list.append(j)
    else:
         if j not in cancer_word_list:
             cancer_word_list.append(i)

In [52]:
keywords =['cancer','chemo','radiotherapy','endocrine','hormonal','immunotherapy','target','biopsy',"immunotherapy", "radiosurgery", "metastasis", "benign", "screening", "exam"
           "guided biopsy", "biopsy complex exam", "biopsy Simple Exam",  "Complex Biopsy", "Simple Biopsy", "Reusable Hot Biopsy Forcep",
            "Biopsy forceps", "3 (13-18 Pcs) Gastric Biopsy", "Specialist Biopsy 1st Specimen Only", "Specialist Biopsy", "Guided Core Biopsy"
            ,"Biopsy", "OGD Biopsy Forcep" "Hot Biopsy forceps" "Disposable Biopsy forceps" ,'Core Biopsy','Radiosurgery','Oncology',
            'Chemotherapy','core','specimen','leukemia' , 'leukaemia' , "filgrastim", "tumour","tumors","tumor", "cytology", "chemotherapy-induced", "Ondansetron"
            ]

keyword = keywords #+ cancer_word_list temp remove this 

lowercase = [str(item).lower() if isinstance(item, str) else item for item in keyword]


In [53]:
lowercase_list = []


for i in lowercase:
    splits = str(i).split()
    if len(splits) > 1:
        for j in splits:
            lowercase_list.append(j)
    else:
        lowercase_list.append(i)



common_words = {'a','b', 'and', 'or', 'the','for', 'with', 'not', 'by', 'ii','iii','system','is', 'of', 'in', 'on', 'at', 'to','1','2','3','4','5','6','7','8','9','12','surgery','1st','2a','3b','c','t3','t4','treatment','type'}

# Input list

# Remove common words from the input list
lowercase_list2 = [word for word in lowercase_list if word not in common_words]




In [54]:
def remove_duplicates(lst):
    seen = set()
    result = []
    for item in lst:
        if item not in seen:
            seen.add(item)
            result.append(item)
    return result


unique_list = remove_duplicates(lowercase_list2)

In [55]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import wordnet
import string
nltk.download('averaged_perceptron_tagger')
# Sample DataFrame
# df_cleaned = pd.DataFrame({'Description': ['Some text about cancer and chemo.', 'Another text mentioning radiotherapy.', 'Description with no keyword.']})

# Assuming df_cleaned is your DataFrame with the 'Description' column


def remove_punctuation(text):
    return ''.join([char for char in text if char not in string.punctuation])


porter = PorterStemmer()
lemmatizer = WordNetLemmatizer()


def process_text(text):
    tokens = word_tokenize(text)
    tokens = [remove_punctuation(token.lower()) for token in tokens if token.isalnum()]  # Remove punctuation and convert to lowercase
    stemmed_tokens = [porter.stem(token) for token in tokens]
    lemmatized_tokens = [lemmatizer.lemmatize(token, get_wordnet_pos(token)) for token in tokens]
    return tokens, stemmed_tokens, lemmatized_tokens


def get_wordnet_pos(token):
    tag = nltk.pos_tag([token])[0][1][0].upper()  # Get the POS tag
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)  # Return NOUN if not found


df_cleaned['Tokens'], df_cleaned['Stemmed'], df_cleaned['Lemmatized'] = zip(*df_cleaned['NLP'].apply(process_text))


def check_keywords(tokens):
    cancerlist = []
    for token in tokens:
        if token in unique_list:
            cancerlist.append(token)
    
    return cancerlist


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/xenang/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
/var/folders/sp/84j13ybj0zd_p4y410zpx7jc0000gn/T/ipykernel_53419/4166229139.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['Tokens'], df_cleaned['Stemmed'], df_cleaned['Lemmatized'] = zip(*df_cleaned['NLP'].apply(process_text))
/var/folders/sp/84j13ybj0zd_p4y410zpx7jc0000gn/T/ipykernel_53419/4166229139.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

In [56]:
len(lowercase_list2)

df = pd.DataFrame(lowercase_list2, columns=['Fruit'])

# Save the DataFrame to an Excel file
df.to_excel('fruits.xlsx', index=False)

In [57]:
df_cleaned['Keywords_Present'] = df_cleaned['Tokens'].apply(check_keywords)
df_cleaned['Keywords_Present2'] = df_cleaned['Stemmed'].apply(check_keywords)
df_cleaned['Keywords_Present3'] = df_cleaned['Lemmatized'].apply(check_keywords)
#df_cleaned['Keywords_Present4'] = df_cleaned['NLP'].apply(check_keywords)
#df_cleaned['Cancer'] = df_cleaned['Keywords_Present'] + df_cleaned['Keywords_Present2'] + df_cleaned['Keywords_Present3'] + df_cleaned['Keywords_Present4']

def is_empty(lst):
    if len(lst) == 0:
        return 0
    else:
        return 1

# Apply the function to create the new column
df_cleaned['is_non_empty'] = df_cleaned['Keywords_Present'].apply(is_empty)
df_cleaned['is_non_empty2'] = df_cleaned['Keywords_Present2'].apply(is_empty)
df_cleaned['is_non_empty3'] = df_cleaned['Keywords_Present3'].apply(is_empty)
#df_cleaned['is_non_empty4'] = df_cleaned['Keywords_Present4'].apply(is_empty)

df_cleaned['Cancer'] = df_cleaned['is_non_empty'] + df_cleaned['is_non_empty2'] + df_cleaned['is_non_empty3'] #+ df_cleaned['is_non_empty4']
columns_to_drop = ['is_non_empty', 'is_non_empty2','is_non_empty3','Tokens','Stemmed','Lemmatized']
df_cleaned = df_cleaned.drop(columns=columns_to_drop)



/var/folders/sp/84j13ybj0zd_p4y410zpx7jc0000gn/T/ipykernel_53419/1055050893.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['Keywords_Present'] = df_cleaned['Tokens'].apply(check_keywords)
/var/folders/sp/84j13ybj0zd_p4y410zpx7jc0000gn/T/ipykernel_53419/1055050893.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['Keywords_Present2'] = df_cleaned['Stemmed'].apply(check_keywords)
/var/folders/sp/84j13ybj0zd_p4y410zpx7jc0000gn/T/ipykernel_53419/1055050893.py:3: SettingWithCopyWarni

In [58]:
# Specify the path for the new Excel file
output_excel_path = 'Output/combined_data.xlsx'
 
# Write the combined dataframe to a new Excel file
df_cleaned.to_excel(output_excel_path, index=False)

# Display a message indicating the success
print(f"Combined data has been written to {output_excel_path}")


Combined data has been written to Output/combined_data.xlsx


In [59]:
breaker

NameError: name 'breaker' is not defined

In [ ]:
salary = pd.read_csv('salary.csv')
salary = salary[salary['xstfws'].notna()]
salary['fyear'] = salary['fyear'].astype(int)

salary.loc[:,'pct_chng'] = salary.groupby('conm')['xstfws'].pct_change().fillna(0) *100
nan_rows = salary[salary['pct_chng'].isna()]
nan_rows
result = salary.groupby(salary['fyear'])['pct_chng'].mean()
salary['value'] = salary.groupby('fyear')['pct_chng'].transform('mean')
salary

,gvkey,fyear,datadate,ceq,xstfws,conm,gsector,pct_chng,value
6,8546,2020,2020-12-31,11870.000,13.000,KONINKLIJKE PHILIPS NV,35,0.000000,2.560941
7,8546,2021,2021-12-31,14438.000,8.000,KONINKLIJKE PHILIPS NV,35,-38.461538,81.816848
8,8546,2022,2022-12-31,13249.000,10.000,KONINKLIJKE PHILIPS NV,35,25.000000,NaN
17,16602,2020,2020-06-30,-24.776,9.136,THE IQ GROUP GLOBAL LTD,35,0.000000,2.560941
18,16602,2021,2021-06-30,-6.960,8.309,THE IQ GROUP GLOBAL LTD,35,-9.052102,81.816848
...,...,...,...,...,...,...,...,...,...
8654,359880,2022,2023-03-31,441.991,86.613,ACCENT MICROCELL LIMITED,35,-6.370398,NaN
8657,360003,2019,2020-03-31,1078.580,202.264,INNOVA CAPTAB LIMITED,35,0.000000,0.000000
8658,360003,2020,2021-03-31,1448.210,223.340,INNOVA CAPTAB LIMITED,35,10.420045,2.560941
8659,360003,2021,2022-03-31,2086.060,404.590,INNOVA CAPTAB LIMITED,35,81.154294,81.816848


In [60]:
import os
hosp_lists = os.listdir("OCR")
hospitalname = []

for i in hosp_lists:
    sep = i.split()
    sep2 = sep[3].split(".")
    hospitalname.append(sep2[0])
hospitalname


['Raffles', 'SGH', 'AALC', 'Parkway', 'NUH', 'TTSH']

In [61]:
import pandas as pd

combined_df = pd.DataFrame()

for hospitals in hosp_lists:
    excel_file_path = "OCR/" + hospitals

    all_sheets = pd.read_excel(excel_file_path, sheet_name=None)
    sep = hospitals.split()
    sep2 = sep[3].split(".")
    hospitalname = sep2[0]


    for sheet_name, sheet_df in all_sheets.items():
        sheet_df['Codes'] = sheet_name
        sheet_df['Hospitals'] = hospitalname
        try:
            combined_df = pd.concat([combined_df, sheet_df], ignore_index=True)
        except ValueError as e:
            print(f"Error processing sheet '{sheet_name}': {e}")


In [62]:
combined_df['Heading'].fillna(combined_df['Code'], inplace=True)

print("check")

check


In [66]:

combined_df['NLP'] = combined_df[['Provider', 'Type', 'Heading','Description','Category','LOG Dx']].apply(concat_columns, axis=1)

combined_df['Tokens'], combined_df['Stemmed'], combined_df['Lemmatized'] = zip(*combined_df['NLP'].apply(process_text))

df_cleaned = combined_df

df_cleaned['Keywords_Present'] = df_cleaned['Tokens'].apply(check_keywords)
df_cleaned['Keywords_Present2'] = df_cleaned['Stemmed'].apply(check_keywords)
df_cleaned['Keywords_Present3'] = df_cleaned['Lemmatized'].apply(check_keywords)
#df_cleaned['Keywords_Present4'] = df_cleaned['NLP'].apply(check_keywords)
#df_cleaned['Cancer'] = df_cleaned['Keywords_Present'] + df_cleaned['Keywords_Present2'] + df_cleaned['Keywords_Present3'] + df_cleaned['Keywords_Present4']

def is_empty(lst):
    if len(lst) == 0:
        return 0
    else:
        return 1

# Apply the function to create the new column
df_cleaned['is_non_empty'] = df_cleaned['Keywords_Present'].apply(is_empty)
df_cleaned['is_non_empty2'] = df_cleaned['Keywords_Present2'].apply(is_empty)
df_cleaned['is_non_empty3'] = df_cleaned['Keywords_Present3'].apply(is_empty)
#df_cleaned['is_non_empty4'] = df_cleaned['Keywords_Present4'].apply(is_empty)

df_cleaned['Cancer'] = df_cleaned['is_non_empty'] + df_cleaned['is_non_empty2'] + df_cleaned['is_non_empty3'] #+ df_cleaned['is_non_empty4']
columns_to_drop = ['is_non_empty', 'is_non_empty2','is_non_empty3','Tokens','Stemmed','Lemmatized']
df_cleaned = df_cleaned.drop(columns=columns_to_drop)



In [67]:
# Specify the path for the new Excel file
output_excel_path = 'Output/OCR_excel.xlsx'

# Write the combined dataframe to a new Excel file
df_cleaned.to_excel(output_excel_path, index=False)

# Display a message indicating the success
print(f"Combined data has been written to {output_excel_path}")


Combined data has been written to Output/OCR_excel.xlsx
